In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [5]:
words = open("names.txt","r").read().split("\n")
words

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn',
 'abigail',
 'emily',
 'elizabeth',
 'mila',
 'ella',
 'avery',
 'sofia',
 'camila',
 'aria',
 'scarlett',
 'victoria',
 'madison',
 'luna',
 'grace',
 'chloe',
 'penelope',
 'layla',
 'riley',
 'zoey',
 'nora',
 'lily',
 'eleanor',
 'hannah',
 'lillian',
 'addison',
 'aubrey',
 'ellie',
 'stella',
 'natalie',
 'zoe',
 'leah',
 'hazel',
 'violet',
 'aurora',
 'savannah',
 'audrey',
 'brooklyn',
 'bella',
 'claire',
 'skylar',
 'lucy',
 'paisley',
 'everly',
 'anna',
 'caroline',
 'nova',
 'genesis',
 'emilia',
 'kennedy',
 'samantha',
 'maya',
 'willow',
 'kinsley',
 'naomi',
 'aaliyah',
 'elena',
 'sarah',
 'ariana',
 'allison',
 'gabriella',
 'alice',
 'madelyn',
 'cora',
 'ruby',
 'eva',
 'serenity',
 'autumn',
 'adeline',
 'hailey',
 'gianna',
 'valentina',
 'isla',
 'eliana',
 'quinn',
 'nevaeh',
 'ivy',
 'sadie',
 'piper',
 'lydia',
 'alexa',
 'josephine',
 'emery',
 'julia'

In [6]:
len(words)

32033

In [27]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(stoi,itos)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0} {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [28]:
block_size = 3

In [29]:
X = []
Y = []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [31]:
C = torch.rand((27,2))
C

tensor([[0.4954, 0.3898],
        [0.1472, 0.5071],
        [0.3964, 0.9915],
        [0.6127, 0.8879],
        [0.7751, 0.3814],
        [0.2972, 0.0407],
        [0.1068, 0.9011],
        [0.0929, 0.7049],
        [0.2600, 0.3705],
        [0.3290, 0.2679],
        [0.1378, 0.1767],
        [0.1386, 0.7307],
        [0.7031, 0.5773],
        [0.5219, 0.8856],
        [0.4617, 0.1776],
        [0.2259, 0.5512],
        [0.4801, 0.0425],
        [0.5770, 0.6798],
        [0.6885, 0.6360],
        [0.6150, 0.3907],
        [0.2577, 0.4180],
        [0.3055, 0.9457],
        [0.4492, 0.4397],
        [0.0201, 0.4823],
        [0.1606, 0.4441],
        [0.1424, 0.3593],
        [0.5937, 0.7920]])

In [54]:
emb = C[X]

In [57]:
w1 = torch.rand(6,100)
b1 = torch.rand(100)

In [60]:
h = torch.tanh(emb.view(emb.shape[0],6) @ w1 + b1)
h.shape

torch.Size([32, 100])

In [61]:
w2 = torch.rand(100,27)
b2 = torch.rand(27)

In [62]:
logits = h @ w2 + b2

In [63]:
counts = logits.exp()
probs = counts / counts.sum(-1, keepdim=True)

In [66]:
probs

tensor([[6.1301e-01, 1.4630e-03, 3.1364e-03, 1.6948e-05, 1.0963e-03, 9.3767e-03,
         1.3883e-01, 3.7239e-03, 2.8849e-02, 2.0865e-05, 2.9247e-04, 1.3562e-03,
         1.6249e-04, 3.2711e-03, 1.6544e-04, 5.3425e-04, 1.7266e-03, 3.8816e-04,
         9.4340e-04, 1.2690e-06, 5.0721e-03, 6.6194e-03, 4.0191e-03, 1.7420e-01,
         6.8356e-06, 1.1629e-03, 5.4807e-04],
        [5.6240e-01, 1.8744e-03, 4.4142e-03, 2.6418e-05, 1.4535e-03, 1.2288e-02,
         1.5349e-01, 4.6947e-03, 3.9173e-02, 3.4546e-05, 5.0116e-04, 1.5534e-03,
         2.5720e-04, 3.3827e-03, 2.3278e-04, 8.0752e-04, 1.9326e-03, 5.1129e-04,
         1.2952e-03, 2.3148e-06, 7.5936e-03, 7.0332e-03, 6.3227e-03, 1.8607e-01,
         1.0712e-05, 1.7017e-03, 9.3084e-04],
        [6.0751e-01, 1.8618e-03, 3.8108e-03, 2.0774e-05, 1.1780e-03, 9.8778e-03,
         1.4363e-01, 3.9611e-03, 3.0032e-02, 2.1000e-05, 2.8909e-04, 1.8233e-03,
         1.8222e-04, 4.1030e-03, 1.8524e-04, 5.8753e-04, 2.0646e-03, 4.6026e-04,
         1.1716e-

In [65]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [69]:
-probs[torch.arange(len(Y)), Y].log().mean()

tensor(6.3881)

In [73]:
F.cross_entropy(logits, Y)

tensor(6.3881)

In [74]:
parameters = [C,w1,b1,w2,b2]

In [75]:
for _ in parameters:
    _.requires_grad = True

In [86]:
epochs = 1000

In [90]:
for i in range(epochs):
    # Forward Pass
    emb = C[X]
    h = torch.tanh(emb.view(emb.shape[0],6) @ w1 + b1)
    logits = h @ w2 + b2
    loss = F.cross_entropy(logits, Y)
    print(loss.item())
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data -= p.grad.data * 1e-1

0.2663688063621521
0.2663463056087494
0.2663238048553467
0.2663014233112335
0.2662791311740875
0.2662568986415863
0.26623478531837463
0.2662126123905182
0.2661905288696289
0.2661685645580292
0.2661466896533966
0.26612475514411926
0.26610296964645386
0.2660812437534332
0.2660595774650574
0.2660379707813263
0.26601642370224
0.26599496603012085
0.2659735679626465
0.2659521698951721
0.26593077182769775
0.2659095525741577
0.2658884525299072
0.2658672630786896
0.2658461332321167
0.26582518219947815
0.26580411195755005
0.2657832205295563
0.2657623589038849
0.26574164628982544
0.2657209038734436
0.26570016145706177
0.26567956805229187
0.265658974647522
0.2656385004520416
0.2656180262565613
0.2655976414680481
0.2655772268772125
0.2655569016933441
0.26553666591644287
0.2655164897441864
0.26549631357192993
0.2654763162136078
0.2654561996459961
0.2654362618923187
0.26541632413864136
0.26539644598960876
0.26537659764289856
0.2653568387031555
0.26533710956573486
0.26531746983528137
0.265297830104827